In [1]:
import os
import requests
from bs4 import BeautifulSoup
import zipfile
import xport, csv
import pandas as pd
from pandas.api.types import infer_dtype
from datetime import datetime
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.select import Select
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from dateutil.relativedelta import relativedelta
import dask.dataframe as dd

Define the urls to extract from and the download folder directory

In [2]:
url1976_2000 = 'https://www.chicagofed.org/banking/financial-institution-reports/commercial-bank-data-complete-1976-2000'
# use selenium for 2001-present
root = 'https://www.chicagofed.org/'
# change the download_folder to your local directory
download_folder = 'C:/Users/kwang648/Downloads/banking/'

define start and end quarter for download

In [3]:
start = '202203' # format:YYYYMM
end = '202303' # format:YYYYMM
start =  datetime.strptime(start, '%Y%m')
end =  datetime.strptime(end, '%Y%m')
quarters = (pd.date_range(start,end + pd.offsets.QuarterBegin(1), freq='Q').strftime('%y%m').tolist())

In [4]:
def extract_links(url_list):
    zip_files = []
    for url in url_list:
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        all_hrefs = soup.find_all('a')
        all_links = [link.get('href') for link in all_hrefs]
        temp = [dl for dl in all_links if '.zip' in dl or '.ZIP' in dl]
        temp = [dl.lower() for dl in temp]
        temp = [dl[:dl.index('.zip')+len('.zip')] for dl in temp]
        for dl in temp:
            zip_files.append(dl)
    return zip_files

In [5]:
def download_zip(zip_files):
    #if download folder does not exist, create one
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    for zip_file in zip_files:
        if re.findall("[0-9]{4}",zip_file)[0] in quarters:
            full_url = root + zip_file
            r = requests.get(full_url)
            zip_filename = os.path.basename(zip_file)
            dl_path = os.path.join(download_folder, zip_filename)
            with open(dl_path, 'wb') as z_file:
                z_file.write(r.content)

In [6]:
#this method is used to extract the links after 2001
def use_selenium(start,end):
    if start >= datetime.strptime('200103', '%Y%m'):
        quarters = (pd.date_range(start, end + pd.offsets.QuarterBegin(1), freq='Q').strftime('%m/%d/%Y').tolist())
    else:
        quarters = (pd.date_range(pd.to_datetime('200103',format='%Y%m'), pd.to_datetime(end) + pd.offsets.QuarterBegin(1), freq='Q').strftime('%m/%d/%Y').tolist())
    # Create Driver Instance
    options = webdriver.ChromeOptions()
    prefs = {"download.default_directory":'C:\\Users\\kwang648\\Downloads\\banking'}
    options.add_experimental_option("prefs",prefs)
    driver = webdriver.Chrome(service=Service(executable_path='C:/Users/kwang648/Downloads/python_code/chromedriver_win32/chromedriver.exe'),options=options)
    url = 'https://cdr.ffiec.gov/public/PWS/DownloadBulkData.aspx'
    driver.get(url)
    #select "call report"
    products = Select(driver.find_element(By.ID,'ListBox1'))
    products.select_by_visible_text('Call Reports -- Single Period')
    #select TSV for the format
    driver.find_element(By.ID,'TSVRadioButton').click()
    for period in quarters:
        #drop down to select dates
        dropdown = Select(driver.find_element(By.XPATH,'//*[@id="DatesDropDownList"]'))
        dropdown.select_by_visible_text(period)
        #click download button
        download_button = driver.find_element(By.ID,"Download_0")
        download_button.click()
        time.sleep(3)

In [7]:
def rename_filename(directory):
    os.chdir(directory)
    for item in os.listdir(directory):
        if bool(re.search('[0-9]{8}.zip', item)):
            new_name = re.findall('[0-9]{8}', item)[0]
            new_name = "call"+datetime.strptime(new_name, '%m%d%Y').strftime('%y%m')+".zip"
            os.rename(item,new_name)
        if bool(re.search('[a-zA-Z]{4}[0-9]{4}.xpt', item)):
            new_name = re.findall('[a-zA-Z]{4}[0-9]{4}.xpt', item)[0]
            os.rename(item,new_name)

In [8]:
def unzip():
    os.chdir(download_folder)
    for item in os.listdir(download_folder):
        if item.endswith('.zip'):
            folder_name = re.search('[a-zA-Z]{4}[0-9]{4}', item)[0]
            if not os.path.exists(download_folder+folder_name):
                os.mkdir(download_folder+folder_name)
            file_name = os.path.abspath(item)
            zip_ref = zipfile.ZipFile(file_name) # create zipfile object
            zip_ref.extractall(download_folder+folder_name) # extract file to dir
            zip_ref.close() # close file
            os.remove(file_name) # delete zipped file
    print('unzip complete')

Download, rename, and unzip the files for each quarter

In [9]:
def download():
    if start <= datetime.strptime('200012', '%Y%m'):
        if end <= datetime.strptime('200012', '%Y%m'):
            download_zip(extract_links([url1976_2000]))
        else:
            download_zip(extract_links([url1976_2000]))
            use_selenium(start,end)
    else:
        use_selenium(start,end)
    print('download zip files complete')

load variable definition

In [10]:
item_code = pd.read_csv('C:/Users/kwang648/Downloads/test.csv')
item_code = item_code.drop(['Notes','Description','citation'],axis=1)
#need to update the currently in use definition to the current quarter
item_code['end'] = item_code['end'].apply(lambda x: 20230630 if x==99991231 else x)
#convert begin and end period to DateTime type
item_code['begin']=pd.to_datetime(item_code['begin'],format='%Y%m%d')
item_code['end']=pd.to_datetime(item_code['end'],format='%Y%m%d')
item_code = item_code.dropna(subset=['var_name'])

extract item code used in the definition

In [11]:
var_list = []
for code in item_code['item_code']:
    list = re.findall("[a-zA-Z0-9]+",code)
    for item in list:
        if item not in var_list:
            var_list.append(item)
var_list.extend(['IDRSSD'])

handle data types

In [12]:
dtype_dict = pd.read_csv('C:/Users/kwang648/Downloads/dtype_dict.csv')
dtype_dict['item_code']=dtype_dict['item_code'].str.upper()
dtype_dict = dict(zip(dtype_dict['item_code'], dtype_dict['dtype']))

Convert text to csv, this function also filters the columns to only keep the columns used in the variable definition

In [13]:
def text_to_csv(item):
    os.chdir(download_folder+item)
    merged = pd.DataFrame()
    for text in os.listdir(download_folder+item):
        if text == 'Readme.txt':
            continue
        curr_list =  pd.read_csv(text, delimiter = "\t",engine='python',on_bad_lines='skip').columns.tolist()
        load_list = [element for element in var_list if element in curr_list]
        if len(load_list)==1:
            continue
        new = pd.read_csv(text, delimiter = "\t",usecols=load_list,engine='python',on_bad_lines='skip')
        new['IDRSSD'] = pd.to_numeric(new['IDRSSD'], errors='coerce')
        new = new.dropna(subset=['IDRSSD'])
        if merged.empty:
            merged = new
        else:
            merged = merged.merge(new,on='IDRSSD',suffixes=('', '_remove'))
            merged = merged.loc[:,~merged.columns.str.contains('Unnamed')]
    merged.drop([i for i in merged.columns if 'remove' in i],axis=1, inplace=True)
    time = int((datetime.strptime(re.findall('[0-9]{4}',item)[0], '%y%m')+relativedelta(day=31)).strftime('%Y%m%d'))
    merged['RSSD9999'] = pd.Series([time for x in range(len(merged.index))],dtype=int)
    merged.rename(columns={'IDRSSD':'RSSD9001'},inplace=True)
    return merged

Convert xpt to csv, this function also filters the columns to only keep the items used in the variable definition

In [14]:
def xpt_to_csv(item):
    os.chdir(download_folder+item)
    for xpt in os.listdir(download_folder+item):
        with open(xpt, 'rb') as f:
            df = xport.to_dataframe(f)
        total_list = df.columns.tolist()
        load_list = [element for element in var_list if element in total_list]
        df = df[[c for c in df.columns if c in load_list]]
        return df

convert and download the csv for each quarter

In [15]:
def convert_download():
    os.chdir(download_folder)
    folder_list = os.listdir(download_folder)
    if not os.path.exists(download_folder+'csv'):
        os.mkdir(download_folder+'csv')
    for item in folder_list:
        if datetime.strptime(re.findall('[0-9]{4}',item)[0], '%y%m') >= datetime.strptime('200103', '%Y%m'):
            new = text_to_csv(item)
        else:
            new = xpt_to_csv(item)
        new.to_csv(download_folder+'csv/'+item+'.csv',index=False)
        print('conversion of '+item+' to csv complete')

merge quarters together

In [16]:
def merge():
    os.chdir(download_folder+'csv/')
    merged = pd.DataFrame()
    first_time=1
    for item in os.listdir(download_folder+'csv/'):
        new = dd.read_csv(item,dtype=dtype_dict)
        if first_time == 1:
            merged = new
            first_time = 0
        else:
            merged = dd.concat([merged,new])
    print('merge quarters complete')
    return merged.compute()

variable definition

In [17]:
def handle_missing_code(code,list):
    code_list = re.findall("[a-zA-Z0-9]+",code)
    for item in code_list:
        if item not in list:
            print(item+' is not available')
            return True
    return False

In [18]:
def variable_definition(call_report):
    call_report.rename(columns={'RSSD9999':'DATE'},inplace=True)
    call_report['DATE']=pd.to_datetime(call_report['DATE'],format='%Y%m%d')
    for varname in item_code['var_name'].unique():
        #find begin and end date
        begin = item_code[item_code['var_name']==varname]['begin']
        end = item_code[item_code['var_name']==varname]['end']
        code = item_code[item_code['var_name']==varname]['item_code']
        #handle variable definition with multiple periods
        i = 1
        while i<=len(begin):
            if handle_missing_code(code.iloc[i-1],call_report.columns.to_list()) ==True:
                i=i+1
                continue
            if len(code.iloc[i-1])==8:
                call_report.loc[(call_report['DATE']>=begin.iloc[i-1]) &(call_report['DATE']<=end.iloc[i-1]),varname] = call_report[(call_report['DATE']>=begin.iloc[i-1]) &(call_report['DATE']<=end.iloc[i-1])][code.iloc[i-1]]
            else:
                call_report.loc[(call_report['DATE']>=begin.iloc[i-1]) &(call_report['DATE']<=end.iloc[i-1]),varname] = call_report[(call_report['DATE']>=begin.iloc[i-1]) &(call_report['DATE']<=end.iloc[i-1])].eval(code.iloc[i-1])
            i=i+1
    call_report.to_csv(download_folder+'call_report.csv',index=False)
    print('Variable definition complete')

initiate functions

In [19]:
download()
rename_filename(download_folder)
unzip()
for item in os.listdir(download_folder):
    rename_filename(download_folder+item)
convert_download()
merged = merge()
variable_definition(merged)

download zip files complete
unzip complete
conversion of call2203 to csv complete
conversion of call2206 to csv complete
conversion of call2209 to csv complete
conversion of call2212 to csv complete
conversion of call2303 to csv complete
merge quarters complete
RSSD9999 is not available
RSSD9348 is not available
RSSD9210 is not available
RCFD1975 is not available
RCON2604 is not available
RCON6645 is not available
RCFD1600 is not available
RCFD1766 is not available
RCFD1400 is not available
RCFD1400 is not available
RCFD1350 is not available
RCFDB987 is not available
RCFD2146 is not available
RCFD0400 is not available
RCFD0390 is not available
RCFD2800 is not available
RCFDB993 is not available
RCFD2850 is not available
RCFD1935 is not available
RCON2350 is not available
RCON2350 is not available
RCON2350 is not available
RCON2604 is not available
RCON2389 is not available
RIAD4000 is not available
RIAD4130 is not available
RIAD4170 is not available
RIADA517 is not available
RIAD4218 i